<h1><center>Breast Cancer Wisconsin (Diagnostic) Data Set</center></h1>
<center> Eduardo Vargas Ferreira

# Introdução

Nesta atividade, vamos aprender os princípio da validação em Machine Learning. Para ilustrar esse processo, utilizaremos o Breast Cancer Wisconsin Data Set, cujo objetivo é identificar as classes benignas ou malignas de câncer de mama.

# Importando as bibliotecas

In [29]:
# ===============================
# Imports de bibliotecas padrão
# ===============================
from typing import List, Dict
import warnings

# Ignorar warnings
warnings.filterwarnings('ignore')

# ===============================
# Imports de bibliotecas de terceiros
# ===============================
import numpy as np
import pandas as pd
import joblib

# ===============================
# Imports do scikit-learn
# ===============================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Exercício: Boas práticas com bibliotecas e dependências

Em projetos de *Machine Learning* é essencial manter uma boa organização dos **imports** e das **dependências** do projeto.  
Essa prática garante clareza no código, reprodutibilidade dos experimentos e facilita a colaboração em equipe.

### Tarefas
1. Pesquise:
   - O que é o arquivo `requirements.txt` e qual é sua função?
   - Por que é recomendável utilizar ambientes virtuais (`venv`, `conda`, `poetry`)?

2. Reflita:  
   - O que poderia acontecer se você não fixar as versões das bibliotecas no seu projeto?


# Lendo os dados e realizando a primeira limpeza na base

Nesta base, cada variável foi convertida em 11 atributos numéricos, com valores variando de 0 a 10. 

In [30]:
base = pd.read_csv('cancer.csv')
base.head()

,Unnamed: 0,Id,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,1,1000025,5,1,1,1,2,1,3,1,1,benign
1,2,1002945,5,4,4,5,7,10,3,2,1,benign
2,3,1015425,3,1,1,1,2,2,3,1,1,benign
3,4,1016277,6,8,8,1,3,4,3,7,1,benign
4,5,1017023,4,1,1,3,2,1,3,1,1,benign


As duas primeiras colunas referem-se às variáveis de identificação e, portanto, não é recomendável que permaneça na base (a menos que sejam realmente necessárias). Caso contrário, corre-se o risco, na pior das hipóteses, de alguma entregar a resposta (que chamamos de variável vazada). Por exemplo, observações com determinada codificação de ID são de referentes à câncer benigno, porque vieram de uma mesma base, ou determinado setor do hospital, etc.

In [31]:
base = base.drop(['Unnamed: 0', 'Id'], axis=1)
print(base.shape)
base.head()

(683, 10)


,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,benign
1,5,4,4,5,7,10,3,2,1,benign
2,3,1,1,1,2,2,3,1,1,benign
3,6,8,8,1,3,4,3,7,1,benign
4,4,1,1,3,2,1,3,1,1,benign


Em geral, é necessário transformar a resposta de interesse, nesse caso a variável `Class`. Isso é feito modificando-a para uma variável dicotômica 0 (benigno) e 1 (maligno). Mas, certifique-se que o modelo que será utilizado exige esse tipo de codificação. Por exemplo, Support Vector Machine, considera a resposta como -1 e 1. 

In [32]:
base['Class'] = np.where(base['Class'] == 'benign', 
                        0, 
                        1)
base.Class.value_counts()

Class
0    444
1    239
Name: count, dtype: int64

# Divisão da base em treino e validação por holdout

O termo holdout vem do fato de "segurarmos" uma parte da base fora do treinamento, de forma a utilizá-la depois, como validação do modelo. Ou seja, a performance é considerada com dados nunca vistos e temos apenas duas partições. 

Começaremos realizando esse processo de forma manual para entendermos com mais profundidade a ideia da técnica. Depois veremos como isso é feito em bibliotecas específicas de Machine Learning. Inicialmente, separamos os dados de treinamento realizando uma amostragem na base original.

In [33]:
base_treino = base.sample(frac=0.7, random_state=42)  # 70% treino
print(base_treino.shape)

(478, 10)


Em seguida, o restante da base é atribuída para validação. Ou seja, o treinamento do modelo e toda a engenharia de característica é feita nos dados de treino. Já os dados de validação servirá para validar se essas mudanças são importantes de fato.

In [34]:
index_not_in_train = base.index.difference(base_treino.index)
base_val = base.loc[index_not_in_train]
print(base_val.shape)

(205, 10)


Agora com as duas bases, dividimos as variáveis explicativas (X) da resposta (y).

In [35]:
X_train = base_treino.drop(['Class'], axis=1)
y_train = base_treino['Class']

X_val = base_val.drop(['Class'], axis=1)
y_val = base_val['Class']

Veja que, manualmente, primeiro dividimos em `base_treino` e `base_val` e depois dividimos em X e y.

# Utilizando holdout para feature scaling

Suponha que, em nosso problema, seja importante padronizar as variáveis antes do treinamento. Isso ocorre, por exemplo, quando utilizamos algoritmos que dependem de distâncias euclidianas, como o K-Nearest Neighbors (KNN).

O procedimento adotado é conhecido como feature scaling, cujo objetivo é colocar todas as variáveis na mesma escala. Uma forma bastante comum de realizar essa transformação é por meio da padronização, definida por:

$$ z = \frac{x-\mu}{\sigma} $$
	​
em que:

- $x$ representa o valor original da variável,

- $\mu$ é a média da variável nos dados de treino,

- $\sigma$ é o desvio padrão da variável nos dados de treino.

Com essa transformação, todas as variáveis passam a ter média igual a zero e variância igual a 1, tornando-as comparáveis em termos de escala.

In [36]:
# Instancia o scaler
scaler = StandardScaler()

# Ajusta nos dados de treino e transforma
X_train_scaled = scaler.fit_transform(X_train)

# Transforma os dados de validação com o mesmo scaler
X_val_scaled = scaler.transform(X_val)

As variáveis do conjunto de treinamento são padronizadas para média zero e variância 1. Os dados de validação, no entanto, não necessariamente terão essas mesmas estatísticas, e isso é intencional.

O objetivo é que qualquer transformação aplicada aos dados futuros seja baseada exclusivamente nas informações aprendidas no treino. Dessa forma, se a padronização apresentar efeito positivo na validação, podemos esperar, com maior segurança, que o mesmo efeito se repetirá em novos dados.

In [37]:
# Cálculo da média e variância para X_train
mean_train = X_train_scaled.mean()
var_train = X_train_scaled.var()

# Exibindo os resultados
print("Média em X_train:\n", mean_train)
print("\nVariância em X_train:\n", var_train)

Média em X_train:
 5.367884451930092e-18

Variância em X_train:
 1.0


In [38]:
# Cálculo da média e variância para X_train
mean_val = X_val_scaled.mean()
var_val = X_val_scaled.var()

# Exibindo os resultados
print("Média em X_val:\n", mean_val)
print("\nVariância em X_val:\n", var_val)

Média em X_val:
 0.018293849460935797

Variância em X_val:
 1.0264096128811269


# Criando funções utilitárias

In [39]:
def fit_scaler_for_numeric(X_train: pd.DataFrame, numeric_cols: List[str] = None) -> Dict:
    """
    Ajusta (fit) um StandardScaler APENAS nas colunas numéricas do treino.
    Retorna um dicionário 'meta' com scaler, colunas numéricas e ordem total das features.
    """
    if numeric_cols is None:
        # Seleciona apenas colunas numéricas
        numeric_cols = X_train.select_dtypes(include=['number', 'bool']).columns.tolist()

    scaler = StandardScaler()
    scaler.fit(X_train[numeric_cols])

    meta = {
        "scaler": scaler,
        "numeric_cols": numeric_cols,
        "feature_order": X_train.columns.tolist(),  # para manter ordem consistente
    }
    return meta


def transform_with_scaler(X: pd.DataFrame, meta: Dict) -> pd.DataFrame:
    """
    Aplica o transform com o scaler treinado nas colunas numéricas.
    Mantém as demais colunas inalteradas e preserva índices/ordem.
    """
    scaler = meta["scaler"]
    numeric_cols = meta["numeric_cols"]
    feature_order = meta["feature_order"]

    # Checagens úteis em produção
    missing = [c for c in numeric_cols if c not in X.columns]
    if missing:
        raise ValueError(f"Faltam colunas numéricas esperadas: {missing}")

    # Copia e transforma apenas numéricas
    X_out = X.copy()
    X_out[numeric_cols] = scaler.transform(X_out[numeric_cols])

    # Reordena as colunas para a mesma ordem do treino (evita surpresas em modelos)
    X_out = X_out.reindex(columns=[c for c in feature_order if c in X_out.columns] + 
                                   [c for c in X_out.columns if c not in feature_order])
    return X_out


def save_scaler_meta(meta: Dict, path: str) -> None:
    """Salva o meta (scaler + infos)."""
    joblib.dump(meta, path)


def load_scaler_meta(path: str) -> Dict:
    """Carrega o meta salvo."""
    return joblib.load(path)

# Aplicando a função

In [40]:
# 'meta' é um dicionário que guarda:
# 1 - O objeto StandardScaler já ajustado com médias e desvios do X_train;
# 2 - A lista de colunas que foram padronizadas;
# 3 - A ordem original das features do treino.  
meta = fit_scaler_for_numeric(X_train)

# Por que guardar a ordem original das *features*?

Quando utilizamos técnicas de **pré-processamento** de dados, como padronização (`StandardScaler`) ou normalização (`MinMaxScaler`), o ajuste (`fit`) do *scaler* é feito com base na ordem das colunas presentes no conjunto de treino.  

Ou seja, o *scaler* aprende estatísticas (como média e desvio padrão) associadas a cada coluna, na ordem exata em que elas estavam no treino.  

Se essa ordem for alterada no futuro, durante a transformação de novos dados (`transform`), os valores poderão ser aplicados incorretamente em colunas diferentes, distorcendo completamente os resultados.  

### Exemplo ilustrativo

Suponha que no treino tínhamos as colunas na ordem:
1. `idade`
2. `altura`
3. `peso`

O *scaler* guardou:
- `idade`: média = 35, desvio = 10  
- `altura`: média = 170, desvio = 7  
- `peso`: média = 70, desvio = 15  

Se no futuro os dados vierem em ordem diferente (por exemplo: `peso`, `idade`, `altura`), o *scaler* aplicará os parâmetros errados em cada coluna. Isso faz com que os dados transformados não façam sentido, prejudicando o modelo.  

Os modelos do scikit-learn esperam arrays, não DataFrames: quando você faz model.fit(X_train, y_train), internamente o modelo enxerga um array NumPy, não os nomes das colunas. Ou seja: o modelo só “sabe” a ordem das colunas que recebeu no treino. Se depois você passar os mesmos dados mas com colunas em ordem diferente, o modelo interpreta errado (coluna A no lugar da B, etc).

In [41]:
scaler = meta["scaler"]

pd.DataFrame({
    "coluna": meta["numeric_cols"],
    "media": scaler.mean_,
    "desvio": scaler.scale_
})

,coluna,media,desvio
0,Cl.thickness,4.453975,2.794378
1,Cell.size,3.100418,3.015712
2,Cell.shape,3.135983,2.905831
3,Marg.adhesion,2.774059,2.785797
4,Epith.c.size,3.175732,2.157617
5,Bare.nuclei,3.535565,3.624330
6,Bl.cromatin,3.456067,2.483661
7,Normal.nucleoli,2.924686,3.115385
8,Mitoses,1.635983,1.816165


# Aplica no treino e na validação (sem refit!)

In [42]:
# TRANSFORM: aplica no treino e na validação (sem refit!)
X_train_std = transform_with_scaler(X_train, meta)
X_val_std   = transform_with_scaler(X_val, meta)

In [43]:
# Cálculo da média e variância para X_train
mean_train = X_train_std.mean()
var_train = X_train_std.var()

# Exibindo os resultados
print("Média em X_train:\n", mean_train)
print("\nVariância em X_train:\n", var_train)

Média em X_train:
 Cl.thickness       1.560816e-16
Cell.size          5.202719e-17
Cell.shape         3.716228e-17
Marg.adhesion     -7.060833e-17
Epith.c.size       3.716228e-17
Bare.nuclei       -3.530416e-17
Bl.cromatin       -5.574342e-17
Normal.nucleoli    0.000000e+00
Mitoses           -7.432455e-17
dtype: float64

Variância em X_train:
 Cl.thickness       1.002096
Cell.size          1.002096
Cell.shape         1.002096
Marg.adhesion      1.002096
Epith.c.size       1.002096
Bare.nuclei        1.002096
Bl.cromatin        1.002096
Normal.nucleoli    1.002096
Mitoses            1.002096
dtype: float64


In [44]:
# Cálculo da média e variância para X_val
mean_val = X_val_std.mean()
var_val = X_val_std.var()

print("\nMédia em X_val:\n", mean_val)
print("\nVariância em X_val:\n", var_val)


Média em X_val:
 Cl.thickness      -0.014079
Cell.size          0.055667
Cell.shape         0.090858
Marg.adhesion      0.067096
Epith.c.size       0.090377
Bare.nuclei        0.008357
Bl.cromatin       -0.014718
Normal.nucleoli   -0.058812
Mitoses           -0.060101
dtype: float64

Variância em X_val:
 Cl.thickness       1.063286
Cell.size          1.108319
Cell.shape         1.187312
Marg.adhesion      1.188553
Epith.c.size       1.200215
Bare.nuclei        1.036073
Bl.cromatin        0.909037
Normal.nucleoli    0.864315
Mitoses            0.697151
dtype: float64


In [17]:
# Salvar o scaler para uso posterior
save_scaler_meta(meta, "scaler_meta.joblib")

# Novas observações

Chegando um novo lote de dados `X_new` com as MESMAS features do treino.

In [45]:
# Carregar meta salvo (em outro processo/tempo)
meta_loaded = load_scaler_meta("scaler_meta.joblib")

# Aplicar apenas o transform:
#X_new_std = transform_with_scaler(X_new, meta_loaded)

# Dividindo usando o sklearn

No sklearn, primeiro se divide em X e y, e depois usamos o `train_test_split` para realizar a partição de treino e validação.

In [46]:
# Separando variáveis independentes (X) e dependente (y)
X = base.drop('Class', axis=1)
y = base['Class']

# Dividindo em treino (70%) e validação (30%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, 
    test_size=0.30, 
    random_state=42
)

# Utilizando holdout para treinar um modelo

Nesta atividade serão omitidas as etapas relacionadas à construção do modelo, como tuning dos parâmetros, pois o foco é somente a validação. Vamos treinar um modelo de classificação. Para isso, utilizamos somente os dados de treinamento (neste caso, os dados já padronizados).

### 1) Split

In [47]:
# 1) Split
X = base.drop('Class', axis=1)
y = base['Class']

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    )


### 2) Fit do scaler apenas no treino

In [48]:
meta = fit_scaler_for_numeric(X_train)

### 3) Transform nas bases (sem refit)

In [49]:
X_train_scaled = transform_with_scaler(X_train, meta)
X_val_scaled   = transform_with_scaler(X_val, meta)

### 4) Treino do KNN

In [50]:
knn = KNeighborsClassifier(
    n_neighbors=5,
    weights="uniform",
    metric="minkowski",
    p=2
)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

### 5) Avaliação

In [51]:
y_val_pred = knn.predict(X_val_scaled)

accuracy_score(y_val, y_val_pred)

0.9609756097560975

In [52]:
confusion_matrix(y_val, y_val_pred)

array([[125,   2],
       [  6,  72]])

# Pipeline completo

In [53]:
# ======================================================================
# Funções de avaliação
# ======================================================================

# função “privada” usada internamente
def _evaluate_classification(y_true, y_pred) -> Dict:
    """Retorna métricas principais de classificação."""
    return {
        "accuracy": float(accuracy_score(y_true, y_pred)),
        "confusion_matrix": confusion_matrix(y_true, y_pred).tolist(),  
    }

# ======================================================================
# Pipeline de treino
# ======================================================================

def train_pipeline(
    base: pd.DataFrame,
    target_col: str = "Class",
    test_size: float = 0.30,
    random_state: int = 42,
    stratify: bool = True,
    # Hiperparâmetros do KNN:
    n_neighbors: int = 5,
    weights: str = "uniform",          # "uniform" ou "distance"
    metric: str = "minkowski",
    p: int = 2                          # p=2 -> distância euclidiana
) -> Dict:
    """
    Treina um pipeline manual:
      1) split treino/val
      2) fit do scaler SOMENTE no treino
      3) transform em treino/val
      4) treino do KNN
      5) avaliação no conjunto de validação

    Retorna um dicionário com artefatos e métricas.
    """
    # 1) Split
    X = base.drop(target_col, axis=1)
    y = base[target_col]

    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=test_size,
        random_state=random_state,
        stratify=y if stratify else None
    )

    # 2) Fit do scaler apenas no treino
    meta = fit_scaler_for_numeric(X_train)

    # 3) Transform nas bases (sem refit)
    X_train_scaled = transform_with_scaler(X_train, meta)
    X_val_scaled   = transform_with_scaler(X_val, meta)

    # 4) Treino do KNN
    knn = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        metric=metric,
        p=p
    )
    knn.fit(X_train_scaled, y_train)

    # 5) Avaliação
    y_val_pred = knn.predict(X_val_scaled)
    metrics = _evaluate_classification(y_val, y_val_pred)

    artifacts = {
        "model": knn,
        "meta": meta,                          # contém scaler, numeric_cols, feature_order
        "feature_order": meta["feature_order"],
        "numeric_cols": meta["numeric_cols"],
        "metrics": metrics,
        "y_val_pred": y_val_pred,
        "y_val_true": y_val.values,
    }
    return artifacts

# ======================================================================
# Pipeline de predição
# ======================================================================

def predict_pipeline(
    model,
    meta: Dict,
    X_new: pd.DataFrame,
    return_proba: bool = False
) -> Dict:
    """
    Aplica o mesmo scaler (já ajustado) e faz predição com o modelo treinado.
    Não há refit aqui.

    return:
      {"y_pred": ..., "y_proba": ... (opcional), "X_new_scaled": ...}
    """
    X_new_scaled = transform_with_scaler(X_new, meta)
    out = {"y_pred": model.predict(X_new_scaled), "X_new_scaled": X_new_scaled}

    if return_proba and hasattr(model, "predict_proba"):
        out["y_proba"] = model.predict_proba(X_new_scaled)
    return out


# Como usar

In [54]:
# 1) Treino + avaliação
art = train_pipeline(
    base=base,
    target_col="Class",
    test_size=0.30,
    random_state=42,
    stratify=True,
    n_neighbors=7,        # ajuste como quiser
    weights="distance",
    p=2                   # euclidiana
)

print("Acurácia de validação:", art["metrics"]["accuracy"])

Acurácia de validação: 0.9560975609756097


In [55]:
# 2) Predição em novos dados
X_new = X_val.iloc[:5].copy()  # pega as 5 primeiras linhas só como exemplo

# Agora você pode rodar a previsão
preds = predict_pipeline(art["model"], art["meta"], X_new, return_proba=True)

y_new = preds["y_pred"]
probas = preds.get("y_proba", None)

print("Predições:", y_new)
print("Probabilidades:", probas)

Predições: [1 1 0 0 0]
Probabilidades: [[0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
